In [2]:
import pandas as pd
from db import load_from_db, save_forecasts_to_db
from prophet import Prophet


c:\Users\gmpal\miniforge3\envs\vpp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [4]:
data_from_db = load_from_db()

In [19]:
for key, value in data_from_db.items():
    if key == "renewables":
        for renewable, renewable_dict in value.items():
            for source_id, source_data in renewable_dict.items():
                df = pd.DataFrame(source_data)
                break
            break



In [14]:
df = df.reset_index()
df.columns = ["ds", "y"]
df["ds"] = df["ds"].dt.tz_localize(None)

model = Prophet()
model.fit(df)

20:41:35 - cmdstanpy - INFO - Chain [1] start processing
20:41:35 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
future = model.make_future_dataframe(periods=24, freq='h')

# Forecast future values
forecast = model.predict(future)

In [20]:
df

,0
time,
2025-01-21 18:31:11.093010+00:00,0.000000
2025-01-21 18:31:12.093010+00:00,0.000000
2025-01-21 18:31:13.093010+00:00,2.991542
2025-01-21 18:31:14.093010+00:00,0.000000
2025-01-21 18:31:15.093010+00:00,0.000000
...,...
2025-01-21 19:11:05.093010+00:00,23.393933
2025-01-21 19:11:06.093010+00:00,23.492962
2025-01-21 19:11:07.093010+00:00,23.519114


In [17]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2025-01-21 18:31:11.093010,0.579121,-0.787321,2.073914,0.579121,0.579121,0.0,0.0,0.0,0.0,0.0,0.0,0.579121
1,2025-01-21 18:31:12.093010,0.578916,-0.901271,2.058595,0.578916,0.578916,0.0,0.0,0.0,0.0,0.0,0.0,0.578916
2,2025-01-21 18:31:13.093010,0.578711,-0.852286,1.966050,0.578711,0.578711,0.0,0.0,0.0,0.0,0.0,0.0,0.578711
3,2025-01-21 18:31:14.093010,0.578506,-0.945061,2.041374,0.578506,0.578506,0.0,0.0,0.0,0.0,0.0,0.0,0.578506
4,2025-01-21 18:31:15.093010,0.578301,-0.889020,1.892127,0.578301,0.578301,0.0,0.0,0.0,0.0,0.0,0.0,0.578301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,2025-01-22 15:11:09.093010,107.473063,-1145.826100,1383.846837,-1145.291841,1384.263446,0.0,0.0,0.0,0.0,0.0,0.0,107.473063
2419,2025-01-22 16:11:09.093010,111.665088,-1199.624895,1505.090888,-1199.851953,1505.394469,0.0,0.0,0.0,0.0,0.0,0.0,111.665088
2420,2025-01-22 17:11:09.093010,115.857114,-1296.642544,1627.532556,-1296.705710,1625.708555,0.0,0.0,0.0,0.0,0.0,0.0,115.857114
2421,2025-01-22 18:11:09.093010,120.049140,-1388.619832,1735.137076,-1391.179467,1736.216235,0.0,0.0,0.0,0.0,0.0,0.0,120.049140


In [ ]:

RENEWABLES = [
    "solar",
    "wind",
]  # scalable for more renewables TODO: avoid redundancy with db.py


# Function to prepare data for Prophet and forecast
def forecast_with_prophet(time_series_df, forecast_steps=24, freq="h"):
    """
    time_series_df: DataFrame with datetime index and one value column.
    forecast_steps: Number of time steps to forecast.
    freq: Frequency string (e.g., 'H' for hourly).

    """
    # Reset index and rename columns for Prophet
    df = time_series_df.reset_index()
    df.columns = ["ds", "y"]

    # ValueError: Column ds has timezone specified, which is not supported. Remove timezone.
    df["ds"] = df["ds"].dt.tz_localize(None)
    print("df")
    print(df.head())
    print(df.shape)
    print(df.tail())
    # Initialize and fit Prophet model
    model = Prophet()
    model.fit(df)

    # Create a future dataframe for forecasting
    future = model.make_future_dataframe(periods=forecast_steps, freq=freq)

    # Forecast future values
    forecast = model.predict(future)
    print("forecast")
    print(forecast.head())
    print(forecast.shape)
    print(forecast.tail())
    return forecast


def forecast_and_save(forecast_steps: int = 24, frequency: str = "S"):
    """
    Data Format:
    data_from_db = {"renewables": {renewable1: {source_id_1: pd.Series, source_id_1: pd.Series,...}, renewable2: {}, ... }, "load": pd.Series, "market_price": pd.Series}

    forecasted = {"renewables": {renewable1: {source_id_1: pd.Dataframe, source_id_1: pd.Dataframe,...}, renewable2: {}, ... }, "load": pd.Dataframe, "market_price": pd.Dataframe}
    """
    

    # same structure as renewables dict
    forecasted = {}
    for key, value in data_from_db.items():
        if key == "renewables":
            forecasted[key] = {}
            for renewable, renewable_dict in value.items():
                forecasted[key][renewable] = {}
                for source_id, source_data in renewable_dict.items():
                    forecasted[key][renewable][source_id] = forecast_with_prophet(
                        source_data, forecast_steps, frequency
                    )
        else:  # forecast wind and solar
            forecasted[key] = forecast_with_prophet(value, forecast_steps, frequency)

    save_forecasts_to_db(forecasted)


if __name__ == "__main__":
    forecast_and_save()
